In [81]:
import pandas as pd
import numpy as np

In [82]:
# LOAD THE DATA AND RENAME THE COLUMN
# ACCOUNT CSV FILE
df = pd.read_csv("../data/accounts.csv")
df.rename(columns={'id': 'account_id'}, inplace=True)
df.rename(columns={'date': 'open_date'}, inplace=True)

In [83]:
# REMOVE SPACES FROM COLUMN NAMES
df.rename(columns=lambda x: x.strip(), inplace=True)

# COUNT THE NUMBER OF MISSING VALUES IN EACH COLUMN; none
missing_values_num = df.isna().sum()
print(missing_values_num)

account_id             0
district_id            0
open_date              0
statement_frequency    0
dtype: int64


In [84]:
# TYPE CAST THE DATE COLUMN
df['open_date'] = pd.to_datetime(df['open_date'])

/var/folders/1z/6nvrnkz13ms1y39fh2328y7m0000gn/T/ipykernel_30024/2923481415.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['open_date'] = pd.to_datetime(df['open_date'])


In [85]:
# LOAD DISTRCICT DATA AND GET THE DISTRICT NAME
district_df = pd.read_csv("./districts_py.csv")

df = pd.merge(df, district_df, on='district_id', how ='left')
df['district_id'] = df['name']

# for i in df.index:
#     for j in district_df.index: # convert it to python function
#         if (df['district_id'][i] == district_df['id'][j]):
#             df['district_id'][i] = district_df['name'][j]

In [86]:
# CHANGE THE COLUMN NAME AND DROP THE COLUMNS
df.rename(columns={'district_id': 'district_name'}, inplace=True)
df = df.iloc[:, :4]

In [87]:
# LOAD THE LINKS DATA FOR NUM_CUSTOMERS COLUMN
link_df = pd.read_csv("../data/links.csv")
temp = pd.DataFrame(link_df.groupby('account_id')['client_id'].count())
temp = temp.rename(columns = {'client_id':'num_customers'})
transpose = temp.T
temp['account'] = transpose.columns

df= df.merge(temp, left_on = "account_id", right_on = "account", how = "left")
df = df.drop("account", axis = 1)

In [88]:
# CREDIT_CARDS
card_df = pd.read_csv("../data/cards.csv")
temp = link_df.merge(card_df, left_on = "id", right_on = "link_id", how = "left")

temp = pd.DataFrame(temp.groupby("account_id")['id_y'].count())
temp = temp.rename(columns = {"id_y": "credit_cards"})

column_extract = temp.T

column_extract
temp['account'] = column_extract.columns

pd.unique(temp['credit_cards'])

array([0, 1])

In [89]:
df = df.merge(temp, left_on = "account_id", right_on="account", how = "left")
df.drop("account", axis = 1, inplace = True)
df

,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards
0,1,Pisek,1995-03-24,monthly,1,0
1,2,Hl.m. Praha,1993-02-26,monthly,2,0
2,3,Kolin,1997-07-07,monthly,2,0
3,4,Pribram,1996-02-21,monthly,1,0
4,5,Cesky Krumlov,1997-05-30,monthly,1,0
...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0
4496,11054,Kromeriz,1996-02-01,weekly,1,0
4497,11138,Karvina,1995-08-18,weekly,1,1
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1


In [97]:
# LOAN
loan_df = pd.read_csv("./loans_py.csv")
# for i in loan_df.index: # convert: .apply(lambda x: ... )
#     if loan_df['status'] == 'A' or loan_df['status'] == 'C':
#         df['loan'] = 'T'
#     else: # loan_df['status'] == 'B' | loan_df['status'] == 'D'
#         df['loan'] = 'F'
df['loan'] = df.apply(lambda x: True if loan_df['status'] == 'A' or loan_df['status'] == 'C' else (False), axis = 1)
df['loan'] = df.apply(lambda x: 'True' if loan_df['status'].isin(['A', 'C']).any() else 'False', axis = 1)
df

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# LOAN_AMOUNT
for i in loan_df.index:
    if df['account_id'] == loan_df['account_id']:
        df['loan_amount'] = loan_df['amount']
        
# LOAN PAYMENT
for i in loan_df.index:
    if df['account_id'] == loan_df['account_id']:
        df['loan_payment'] = loan_df['payments']

In [ ]:
# LOAN_TERM
if df['loan'] == 'T':
    df['loan_term'] = loan_df['terms']

# LOAN_STATUS
if df['loan'] == 'T':
    if loan_df['status'] == 'A' or loan_df['status'] == 'B':
        df['loan_status'] = 'expired'
    else: # C or D
        df['loan_status'] = 'current'

In [ ]:
# LOAN_DEFAULT:
if loan_df['status'] == 'B' or loan_df['status'] == 'D':
    df['loan_default'] = 'T'
else:
    df['loan_default'] = 'F'

In [ ]:
# MAX_WITHDRAWAL
